# Optuna pipeline

Цель данного ноутбука -- показать, как можно применять библиотеку `optuna` для оптимизации гиперпараметров и подбора элементов пайплайна. Начало скопировано из ноутбука про `hyperopt`.

Главная [страница](https://optuna.org/) библиотеки. [Документация](https://optuna.readthedocs.io/en/stable/index.html).
Судя по документации, это довольно интересная штука с большими возможностями по настройке, сохранению и обработке результатов экспериментов.
Интересные страницы, которые стоит посмотреть:
 - [optuna.study](https://optuna.readthedocs.io/en/stable/reference/study.html)
 - [Study](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.Study.html#optuna.study.Study)
 - [Trial](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html#optuna.trial.Trial)
 - [как настроить verbose](https://optuna.readthedocs.io/en/stable/reference/logging.html)

Далее идет кусок из ноута по `hyperopt`. Сделаю его позднее более коротким, но пока есть что есть :)

---

## 1. Technicals

In [1]:
import time
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import umap

from sklearn import datasets, metrics, model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline

from hyperopt import hp
# from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.manifold import Isomap
from sklearn.decomposition import PCA
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

import optuna
import lightgbm as lgb
import xgboost as xgb
# import catboost as ctb

# новый пакет!
from feature_engine.encoding import WoEEncoder
from feature_engine.creation import CombineWithReferenceFeature
from feature_engine.encoding import OneHotEncoder

from typing import List, Union
from feature_engine.encoding.base_encoder import BaseCategoricalTransformer
from feature_engine.validation import _return_tags
from feature_engine.variable_manipulation import _check_input_parameter_variables

from mlxtend.feature_selection import SequentialFeatureSelector
from feature_engine.selection  import SelectByShuffling
from feature_engine.selection  import RecursiveFeatureAddition
from feature_engine.selection  import SmartCorrelatedSelection

In [2]:
seed = 42

def Gini(y, y_pred):
    res = roc_auc_score(y, y_pred) * 2 - 1
    print(f"Gini: {res}")
    return(res)

## 2. Import Dataset

In [3]:
X_train = pd.read_parquet('../datasets/01_german/samples/X_train.parquet')
y_train = pd.read_parquet('../datasets/01_german/samples/y_train.parquet').target

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_test  = pd.read_parquet('../datasets/01_german/samples/X_test.parquet')
y_test  = pd.read_parquet('../datasets/01_german/samples/y_test.parquet').target

In [4]:
with open('../datasets/01_german/factors.json') as json_file:
    factors_dict = json.load(json_file)

factors_dict['cat_vals']

['cheq_acc',
 'cred_hist',
 'purp',
 'save_acc',
 'empl_t',
 'pers_status',
 'guarant_flg',
 'prop',
 'inst_plan',
 'house',
 'job',
 'tel_flg',
 'foreign_flg']

## 3. Define Modules

All the modules that might be part of the pipeline should be defined below (or import them):

In [12]:
class CombineWithReferenceFeature_adj():
    """
    Обертка вокруг CombineWithReferenceFeature()
    Позволяет не устанавливать параметры
    + variables_to_combine
    + reference_variables
    заранее (иначе не будет работать с OneHotEncoder
    и прочими преобразователями данных, а делать это при .fit()
    """
    def __init__(self, operations):
        self.operations = operations
        
    def fit(self, X, y):
        self.combinator = CombineWithReferenceFeature(
            variables_to_combine = list(X.columns),
            reference_variables = list(X.columns),
            operations = self.operations
        )
        self.combinator.fit(X, y)
        return(self)
    
    def transform(self, X):
        return(self.combinator.transform(X))        

In [13]:
class DimensionReducer():
    """
    Обертка нужна:
    1. Чтобы не заменять фичи, а добавлять их к исходному df
    2. Для PCA ouput = np.array, требуется заменить на pd.DataFrame 
    """
    def __init__(self, gen_class, **kwargs):
        self.reducer = gen_class(**kwargs)
        # self.reducer.set_params()
        
    def fit(self, X, y):
        self.reducer.fit(X, y)
        return self
    
    def transform(self, X):
        # potentially 
        return pd.concat([X, pd.DataFrame(self.reducer.transform(X), index = X.index)], axis=1)
    
    def set_params(self, **kwargs):
        self.reducer.set_params(**kwargs)
        return self     

In [14]:
# from typing import List, Union

# import numpy as np
# import pandas as pd

# from feature_engine.encoding.base_encoder import BaseCategoricalTransformer
# from feature_engine.validation import _return_tags
# from feature_engine.variable_manipulation import _check_input_parameter_variables

class WoEEncoder_adj(BaseCategoricalTransformer):
    def __init__(
        self,
        variables: Union[None, int, str, List[Union[str, int]]] = None,
        ignore_format: bool = False,
    ) -> None:

        if not isinstance(ignore_format, bool):
            raise ValueError("ignore_format takes only booleans True and False")

        self.variables = _check_input_parameter_variables(variables)
        self.ignore_format = ignore_format

    def fit(self, X: pd.DataFrame, y: pd.Series):
        """
        Learn the WoE.
        Parameters
        ----------
        X: pandas dataframe of shape = [n_samples, n_features]
            The training input samples.
            Can be the entire dataframe, not just the categorical variables.
        y: pandas series.
            Target, must be binary.
        """
        
        X = self._check_fit_input_and_variables(X)

        if not isinstance(y, pd.Series):
            y = pd.Series(y)

        # check that y is binary
        if y.nunique() != 2:
            raise ValueError(
                "This encoder is designed for binary classification. The target "
                "used has more than 2 unique values."
            )

        temp = pd.concat([X, y], axis=1)
        temp.columns = list(X.columns) + ["target"]

        # if target does not have values 0 and 1, we need to remap, to be able to
        # compute the averages.
        if any(x for x in y.unique() if x not in [0, 1]):
            temp["target"] = np.where(temp["target"] == y.unique()[0], 0, 1)

        self.encoder_dict_ = {}

        total_pos = temp["target"].sum()
        total_neg = len(temp) - total_pos
        temp["non_target"] = np.where(temp["target"] == 1, 0, 1)

        for var in self.variables_:
            pos = (temp.groupby([var])["target"].sum() + .5) / total_pos
            neg = (temp.groupby([var])["non_target"].sum() + .5) / total_neg

            t = pd.concat([pos, neg], axis=1)
            t["woe"] = np.log(t["target"] / t["non_target"])

            # we make an adjustment to override this error
            # if (
            #     not t.loc[t["target"] == 0, :].empty
            #     or not t.loc[t["non_target"] == 0, :].empty
            # ):
            #     raise ValueError(
            #         "The proportion of one of the classes for a category in "
            #         "variable {} is zero, and log of zero is not defined".format(var)
            #     )

            self.encoder_dict_[var] = t["woe"].to_dict()

        self._check_encoding_dictionary()

        self.n_features_in_ = X.shape[1]

        return self

    # Ugly work around to import the docstring for Sphinx, otherwise not necessary
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = super().transform(X)

        return X

    transform.__doc__ = BaseCategoricalTransformer.transform.__doc__

    def inverse_transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = super().inverse_transform(X)

        return X

    inverse_transform.__doc__ = BaseCategoricalTransformer.inverse_transform.__doc__

    def _more_tags(self):
        tags_dict = _return_tags()
        # in the current format, the tests are performed using continuous np.arrays
        # this means that when we encode some of the values, the denominator is 0
        # and this the transformer raises an error, and the test fails.
        # For this reason, most sklearn transformers will fail. And it has nothing to
        # do with the class not being compatible, it is just that the inputs passed
        # are not suitable
        tags_dict["_skip_test"] = True
        return tags_dict

In [15]:
WoE_module = WoEEncoder_adj(variables = factors_dict['cat_vals'])

OneHot_module = OneHotEncoder(variables = factors_dict['cat_vals'])

PCA_module = DimensionReducer(
    gen_class = sklearn.decomposition.PCA,
    n_components = 2,    # сколько оставить компонентов; по дефолту - все
    whiten = False,      # отключаем whitening - декорреляцию фичей
    svd_solver = "full", # детали SVD преобразования, за подробностями см. доки
)

kPCA_module = DimensionReducer(
    gen_class = sklearn.decomposition.KernelPCA,
    n_components = 8,  # сколько оставить компонентов; по дефолту - все
    kernel = "linear", # ядро. По дфеолту линейное. Можно сделать своё, но тогда его нужно предварительно вычислить отдельно,
                       # поставить kernel = "precomputed" и передать уже вычисленное ядро в качестве X
    degree = 3,        # степень полинома для некоторых типов ядер. Важный параметр для тьюнинга, но сильно напрягает процессор
    n_jobs = -1        # объект умеет быть многопоточным! -1 займет все ядра
)

Isomap_module = DimensionReducer(
    gen_class = sklearn.manifold.Isomap,
    n_neighbors = 5, #количество соседей при вычислении KNN. Основной гиперпараметр, кстати (!!!)
    n_components = 2,  #сколько оставить компонент; по дефолту - 2
    path_method = "auto", #алгоритм, который вычисляет кратчайший путь. Варианты см. на странице функции. Этот подбирает сам.
    neighbors_algorithm = "auto", #алгоритм, который ищет соседей. Инстанс класса NearestNeighbours
    n_jobs = -1 #объект умеет быть многопоточным! -1 займет все ядра
)

UMAP_module = DimensionReducer(
    gen_class = umap.UMAP,
    n_neighbors = 5,  # количество соседей при вычислении KNN. Основной гиперпараметр, кстати (!!!)
    n_components = 2, # сколько оставить компонентов; по дефолту - 2
    min_dist = 0.1    # минимальная дистанция, которую можно сохранять между точками в получающемся пространстве. 
    # Гиперпараметр. При увеличении начинает лучше улавливать общую структуру, но хуже - локальную
)

CombWRef_module = CombineWithReferenceFeature_adj(
    operations = ['mul']
)

lgbm_mdl = LGBMClassifier(
    num_leaves = 10,
    learning_rate = .1,
    reg_alpha = 8,
    reg_lambda = 8,
    random_state = seed
)

# Tackling imbalances in target
RUS_module    = RandomUnderSampler(random_state = seed)
ROS_module    = RandomOverSampler(random_state = seed)
SMOTE_module  = SMOTE(random_state = seed)
ADASYN_module = ADASYN(random_state = seed)

# feature selection
SeqFearSel_module = SequentialFeatureSelector(
    estimator  = lgbm_mdl,  
    # k_features = 5,                                                  
    forward    = True,                                                  
    floating   = True,                                                
    verbose    = 0,
    cv         = 5
)
RecFeatAdd_module = RecursiveFeatureAddition(
    lgbm_mdl,
    threshold = 0.005
)
# SelShuffl_module = SelectByShuffling(
#     estimator = lgbm_mdl,
#     # variables=X.columns.to_list(),                                      # можно задать подмножество
#     scoring='roc_auc',                                                  # метрика
#     threshold=0.01,                                                     # порог ее снижения
#     cv=5,
#     random_state=42
# )
SmartSel_module = SmartCorrelatedSelection(
    # variables=X.columns.to_list(),
    method="pearson",                # можно взять свою функцию
    threshold=0.3,                   # порог корреляции
    selection_method="variance",     # из коррелирующих групп выбираем признак с наиб дисперсией
    estimator=None,                  # понадобится для selection_method="model_performance"        
    cv=5
)


---

Далее начинается мой код:

In [16]:
modules = {
    'WoE':         WoE_module,
    'OneHot':      OneHot_module,
    'PCA':         PCA_module,
    'kPCA':        kPCA_module,
    'Isomap':      Isomap_module,
    'UMAP':        UMAP_module,
    'CombWRef':    CombWRef_module,
    'RecFeatAdd':  RecFeatAdd_module,
    'lgbm':        lgbm_mdl,
    'RUS':         RUS_module,      
    'ROS':         ROS_module,      
    'SMOTE':       SMOTE_module,  
    'ADASYN':      ADASYN_module,
    'SeqFearSel':  SeqFearSel_module,
    'RecFeatAdd':  RecFeatAdd_module,
    # 'SelShuffl':   SelShuffl_module,
    'SmartSel':    SmartSel_module,
    'passthrough' : 'passthrough'      # можно добавить в (не последнюю) ячейку пайплайна, чтобы пропустить ее
}

## 4. Define Pipeline

### 4.1. Структура Pipeline

Определим структуру самого пайплайна. Словесное описание:
    
1. Энкодинг категориальных переменных:
    + OneHotEncoder
    + WoE
3. Feature Engineering:
    + PCA
    + Kernel PCA
    + Isomap
    + UMAP
    + Combine with Reference (feature multiplication)
    + _отсутствует_
4. Feature Selection:
    + RecursiveFeatureAddition
    + SequentialFeatureSelector
    + SmartCorrelatedSelection
    + _отсутствует_
4. Resampling:
    + Randomised Undersampling (RUS)
    + Randomised Oversampling  (ROS)
    + Synthetic Minority Oversampling Technique (SMOTE)
    + Adaptive Synthetic (ADASYN)
    + _отсутствует_
5. LightGBM

Кандидаты для каждого элемента пайплайна (`passthrough` означает, что пропускаем).

In [6]:
pipe_params = {
    # 'missing_vals': 
    'cat_encoding':  ['OneHot', 'WoE'], # , 'woe' пропустить нельзя из-за наличия кат. пер-х
    'imbalance':     ['passthrough', 'RUS', 'ROS', 'SMOTE', 'ADASYN'],
    'feat_eng':      ['passthrough', 'PCA', 'kPCA', 'Isomap', 'UMAP'], # , 'CombWRef' # удалил, т.к. долго считается
    'feat_sel':      ['passthrough', 'SeqFearSel', 'RecFeatAdd', 'SmartSel'], # 'SelShuffl' is omitted, since it might drop all Xs
    'lgbm':          ['lgbm']
}

Функции для выбора гиперпараметров для отдельных элементов:

In [7]:
def PCA_params(trial) -> dict:
    return {
        "n_components" : trial.suggest_int("PCA__n_components", 2, 11),
        "whiten" : trial.suggest_categorical(
            "PCA__whiten",
            [True, False]
            ),
        "svd_solver" : trial.suggest_categorical(
            "PCA__svd_solver",
            ['full', 'arpack', 'auto', 'randomized']
            )
    }

def kPCA_params(trial):
    return {
        "n_components" : trial.suggest_int("kPCA__n_components", 5, 11),
        "kernel" : trial.suggest_categorical(
            "kPCA__kernel",
            ['linear', 'poly', 'rbf', 'sigmoid', 'cosine', 'precomputed']
            ),
    }

def Isomap_params(trial):
    return {
        "n_neighbors" : trial.suggest_int("Isomap__n_neighbors", 2, 11),
        "n_components" : trial.suggest_int("Isomap__n_components", 2, 5),
        "path_method" : trial.suggest_categorical(
            "Isomap__path_method",
            ['auto', 'FW', 'D']
            )
    }

def UMAP_params(trial):
    return {
        "n_neighbors" : trial.suggest_int("UMAP__n_neighbors", 2, 11),
        "n_components" : trial.suggest_int("UMAP__n_components", 2, 11),
        "min_dist" : trial.suggest_float("UMAP__min_dist", 0.05, 1, step=0.05)
    }

def LightGBM_params(trial):
    return {
        "learning_rate" : trial.suggest_float("LightGBM__learning_rate", 0.05, 0.31, step=0.05),
        "num_leaves" : trial.suggest_int("LightGBM__num_leaves", 5, 16),
        "reg_alpha" : trial.suggest_int("LightGBM__reg_alpha", 0, 16),
        "reg_lambda" : trial.suggest_int("LightGBM__reg_lambda", 0, 0.16),
        "n_estimators" : 100
    }

Сложим их в словарик и подадим его на вход оптимизируемой функции:

In [8]:
modules_hparams = {
    'PCA' : PCA_params,
    'kPCA' : kPCA_params,
    'UMAP' : UMAP_params,
    'LightGBM' : LightGBM_params
}

Оптимизиуемая функция. Должна принимать на вход объект `trial` и возвращать значение метрики:

In [9]:
def objective(trial, modules, pipe_params, modules_hparams):
    pipeline = []
    
    # выбираем модули и параметры к ним
    for elem, options in pipe_params.items():                                    # итерация по парам (ключ, значение) словаря
        choice : str = trial.suggest_categorical(f"pipe__{elem}", options)       # выбираем элемент пайплайна
        module = modules[choice]                                                 # достаем этот объект из словаря

        if choice in modules_hparams.keys():                                     # если мы также оптимизируем гиперпараметры для элемента пайплайна
            hp_function : callable = modules_hparams[choice]                     # функция, которая возвращает гиперпараметры
            hp_choice : dict = hp_function(trial)                                # конкретный набор, который она вернула
            module.set_params(**hp_choice)

        pipeline.append(
            (choice, module)
        )
    
    pipeline = Pipeline(pipeline)                                                # получаем модель

    # далее как-то оцениваем качество получившейся модели
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)[:, 1]
    metric = roc_auc_score(y_test, y_pred)

    return metric

Собственно запуск оптимизации:

In [17]:
study = optuna.create_study(direction='maximize')
study.optimize(
    func=lambda trial: objective(trial, modules, pipe_params, modules_hparams),  # оптимизируемая функция
    n_trials=5,                                                                  # число попыток
    timeout=600                                                                  # лимит по времени
    )

[I 2022-01-09 21:48:32,529] A new study created in memory with name: no-name-0ca632e5-a60a-4c8d-8a04-1db70d88949f
[I 2022-01-09 21:48:36,342] Trial 0 finished with value: 0.7811636013943983 and parameters: {'pipe__cat_encoding': 'WoE', 'pipe__imbalance': 'SMOTE', 'pipe__feat_eng': 'PCA', 'PCA__n_components': 5, 'PCA__whiten': False, 'PCA__svd_solver': 'arpack', 'pipe__feat_sel': 'RecFeatAdd', 'pipe__lgbm': 'lgbm'}. Best is trial 0 with value: 0.7811636013943983.
[I 2022-01-09 21:48:57,074] Trial 1 finished with value: 0.6642625315542733 and parameters: {'pipe__cat_encoding': 'WoE', 'pipe__imbalance': 'RUS', 'pipe__feat_eng': 'UMAP', 'UMAP__n_neighbors': 8, 'UMAP__n_components': 9, 'UMAP__min_dist': 0.55, 'pipe__feat_sel': 'passthrough', 'pipe__lgbm': 'lgbm'}. Best is trial 0 with value: 0.7811636013943983.
[I 2022-01-09 21:49:08,782] Trial 2 finished with value: 0.7622911407621109 and parameters: {'pipe__cat_encoding': 'OneHot', 'pipe__imbalance': 'ADASYN', 'pipe__feat_eng': 'passthrou

In [18]:
study.best_params

{'pipe__cat_encoding': 'WoE',
 'pipe__imbalance': 'SMOTE',
 'pipe__feat_eng': 'PCA',
 'PCA__n_components': 8,
 'PCA__whiten': False,
 'PCA__svd_solver': 'auto',
 'pipe__feat_sel': 'RecFeatAdd',
 'pipe__lgbm': 'lgbm'}

In [19]:
study.best_trial

FrozenTrial(number=4, values=[0.7854309412188965], datetime_start=datetime.datetime(2022, 1, 9, 21, 49, 15, 357238), datetime_complete=datetime.datetime(2022, 1, 9, 21, 49, 19, 436425), params={'pipe__cat_encoding': 'WoE', 'pipe__imbalance': 'SMOTE', 'pipe__feat_eng': 'PCA', 'PCA__n_components': 8, 'PCA__whiten': False, 'PCA__svd_solver': 'auto', 'pipe__feat_sel': 'RecFeatAdd', 'pipe__lgbm': 'lgbm'}, distributions={'pipe__cat_encoding': CategoricalDistribution(choices=('OneHot', 'WoE')), 'pipe__imbalance': CategoricalDistribution(choices=('passthrough', 'RUS', 'ROS', 'SMOTE', 'ADASYN')), 'pipe__feat_eng': CategoricalDistribution(choices=('passthrough', 'PCA', 'kPCA', 'Isomap', 'UMAP')), 'PCA__n_components': IntUniformDistribution(high=11, low=2, step=1), 'PCA__whiten': CategoricalDistribution(choices=(True, False)), 'PCA__svd_solver': CategoricalDistribution(choices=('full', 'arpack', 'auto', 'randomized')), 'pipe__feat_sel': CategoricalDistribution(choices=('passthrough', 'SeqFearSel'

In [20]:
study.best_value

0.7854309412188965